# World Bank Data

In this small exercise, we will look at the World Bank dataset, which can be found here: http://jsonstudio.com/resources/.

Specifically, we will perform the following tasks:

1. Find the 10 countries with the most projects
2. Find the top 10 major project themes (in the 'mjtheme_namecode' column)
3. In the 'mjtheme_namecode' column, some entries contain only the numeric code of the project theme, and are missing the name. So we will create a dataframe with the missing names filled in.

In [1]:
#Import libraries
import pandas as pd
import json
from pandas.io.json import json_normalize

In [2]:
#Read in json data as a dataframe
json_df = pd.read_json('world_bank_projects.json')

<b> 1. Find the 10 countries with the most projects <b/> 

In [3]:
#Display the top 10 countries with the most projects
countries = json_df['countryshortname']
countries.value_counts().head(10)

China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Morocco               12
Bangladesh            12
Nepal                 12
Mozambique            11
Africa                11
Name: countryshortname, dtype: int64

The top 10 countries with the most projects are displayed above. However,
we see that this list contains Africa, which is not a country, and since we don't 
know what the specific African countries are, we can choose 
to exclude Africa from this list.

In [4]:
#Display the top 10 countries after removing Africa from the list
countries[countries != 'Africa'].value_counts().head(10)

China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Morocco               12
Bangladesh            12
Nepal                 12
Mozambique            11
Brazil                 9
Name: countryshortname, dtype: int64

<b>2. Find the top 10 major project themes (using the 'mjtheme_namecode' column) <b/>

In [5]:
#load json as a string
projects = json.load((open('world_bank_projects.json')))

In [6]:
#normalize the json string into a dataframe
proj_themes = json_normalize(projects, 'mjtheme_namecode')
proj_themes['name'].value_counts()[0:10]

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

<b>3. In the 'mjtheme_namecode' column, some entries are missing the project theme name. Create a dataframe with the missing names filled in. <b/>

In [7]:
#A quick inspection confirms that some values in the 'name' column are missing
proj_themes[['code', 'name']]

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion


In [8]:
'''Creates a dataframe with the unique code-name pairing, which we can use to look up the appropriate name values'''

code_names = proj_themes[proj_themes['name'] != ''].drop_duplicates()
code_names

,code,name
0,8,Human development
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
8,7,Social dev/gender/inclusion
11,4,Financial and private sector development
18,10,Rural development
53,9,Urban development


In [9]:
'''The following loops through each entry of the 'name' column in the proj_themes dataframe, 
and fills in any blanks by looking up the appropriate value in the code_names dataframe defined above'''

for i in range(len(proj_themes['name'])):
    if proj_themes['name'][i] != '':
        continue
    proj_themes['name'][i] = code_names[code_names['code'] == proj_themes['code'][i]].reset_index(drop = 1)['name'][0]  

In [10]:
#A quick look at the new dataframe shows that the blanks have been filled in
proj_themes

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion


In [11]:
#Dropping duplicate rows of the new dataframe shows that the blanks were filled in correctly
proj_themes.drop_duplicates()

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
8,7,Social dev/gender/inclusion
11,4,Financial and private sector development
18,10,Rural development
53,9,Urban development
